In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import requests

# Fetch data from your Flask API
api_url = 'http://127.0.0.1:5000/controller_youtube_channel/api/youtube_channel_data'
response = requests.get(api_url)
youtube_data = response.json()

# Convert the data into a DataFrame
youtube_df = pd.DataFrame(youtube_data)

# Calculate the percentage values for each metric
for col in ['subscribers', 'videos', 'likes', 'comments', 'views']:
    youtube_df[f'{col} (%)'] = (youtube_df[col] / youtube_df[col].sum()) * 100

# Sample data for the second chart (replace with your own data)
data_reddit = pd.DataFrame({
    'Upload Time': pd.to_datetime(['2023-10-01 08:00:00', '2023-10-01 10:00:00', '2023-10-01 12:00:00', '2023-10-01 14:00:00', '2023-10-01 16:00:00']),
    'Views': [100, 150, 200, 250, 300],
    'Comments': [10, 15, 20, 25, 30]
})

# Create a Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.Div([
        html.H1("Pizza Hut Social Media Data Dashboard"),
        dcc.Graph(id='multi-bar-charts'),
    ], style={'width': '50%', 'display': 'inline-block'}),
    html.Div([
        dcc.Tabs(id='tabs', value='Reddit', children=[
            dcc.Tab(label='Reddit', value='Reddit'),
            dcc.Tab(label='Source2', value='Source2'),
        ]),
        html.Div(id='tabs-content'),
    ], style={'width': '50%', 'display': 'inline-block'}),
], style={'background-color': 'lightgray', 'color': 'black'})  # Set background and text color for the main layout

# Define a callback to update the multi-bar charts
@app.callback(
    Output('multi-bar-charts', 'figure'),
    Input('multi-bar-charts', 'relayoutData')
)
def update_multi_bar_charts(_):
    traces = []
    
    for col in ['subscribers (%)', 'videos (%)', 'likes (%)', 'comments (%)', 'views (%)']:
        trace = go.Bar(
            x=youtube_df['country'],
            y=youtube_df[col],
            name=col,
        )
        traces.append(trace)

    layout = go.Layout(
        barmode='group',
        title='YouTube Channel Analytics by Country (Percentage)',
        xaxis={'title': 'Country'},
        yaxis={'title': 'Percentage (%)'},
        template='plotly_dark'
    )
    
    fig = go.Figure(data=traces, layout=layout)

    return fig

# Define a callback to update the line chart for Reddit and Source2
@app.callback(
    Output('tabs-content', 'children'),
    [Input('tabs', 'value')]
)
def render_content(tab):
    if tab == 'Reddit':
        data = data_reddit
    elif tab == 'Source2':
        data = data_source2  # Replace with your actual data

    fig = px.line(data, x='Upload Time', y=['Views', 'Comments'], title=f'{tab} Stats')
    fig.update_layout(
        xaxis_title='Upload Time',
        yaxis_title='Count',
    )

    return dcc.Graph(figure=fig)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
